# Import Packages

In [13]:
# we will import everything here
import numpy as np

from src.read_file import *

%reload_ext autoreload 
%autoreload 2 

In [15]:
M = read_file("input.txt")
print(type(M))
print(M)


<class 'numpy.ndarray'>
[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 1 2 3 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
